## ***Step 0:*** Import the Necessary

0.1 Install the necessary

In [ ]:
!pip install -U sentence-transformers
!pip install nltk
!pip install sklearn
!pip install transformers
!pip install torch
!pip install tensorflow
!pip install torchinfo

Requirement already up-to-date: sentence-transformers in /home/evi/anaconda3/lib/python3.7/site-packages (2.2.0)


0.2 Import Libraries

In [ ]:
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import nltk
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import torch
import io
import random
import string # to process standard python strings
from sklearn.feature_extraction.text import TfidfVectorizer
import json
from pathlib import Path
from torch.utils.data import DataLoader
import time

0.3 Usefull Info

1.https://theaidigest.in/how-to-do-semantic-document-similarity-using-bert/com/geekculture/simple-chatbot-using-bert-and-pytorch-part-1-2735643e0baa
2.https://towardsdatascience.com/bert-nlp-how-to-build-a-question-answering-bot-98b1d1594d7b
3.https://pretagteam.com/question/how-to-use-bert-for-document-similarity-of-this-site
4.https://www.kite.com/python/answers/how-to-read-a-text-file-into-a-list-in-python
5.https://reposhub.com/python/deep-learning/MartinoMensio-spacy-sentence-bert.html
7.https://pypi.org/project/sentence-transformers/0.3.2/
6.https://stackoverflow.com/questions/52074153/cannot-convert-list-to-array-valueerror-only-one-element-tensors-can-be-conver   


## ***Step 1:*** Download SQuAD 2.0 dataset 

The ***Stanford Question Answering Dataset*** (SQuAD) consists of questions (100,000)  on a set of Wikipedia articles where the answer to every question is a segment of text, or span, from the corresponding reading passage. In second version of SQuAD unanswerable questions were added (50,000) to look similar to answerable ones.   

https://rajpurkar.github.io/SQuAD-explorer/explore/v2.0/dev/

In [ ]:
%%capture
!mkdir squad
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json -O squad/train-v2.0.json
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json -O squad/dev-v2.0.json

## ***Step 2:*** Retrieve and Store the data

2.1 Store the texts, queries and answers of SQUAD dataset from the train and validation .json files. Informations are saved into lists.
Training data: It has the input data together with correct/expected output; This dataset is usually duly prepared either by humans or by collecting some data in a semi-automated way. But you must have the expected output for every data row here because you need this for supervised learning.

In [ ]:
# Give the path for train data
path = Path('squad/train-v2.0.json')

# Open .json file
with open(path, 'rb') as f:
    squad_dict = json.load(f)

texts = []


# Search for each passage, its question and its answer
for group in squad_dict['data']:
    for passage in group['paragraphs']:
        context = passage['context']
        texts.append(context)

train_texts = texts

2.2 Open, process and store data file from PubMed

In [ ]:
my_file = open("myfile", "r")
content = my_file.read()
content = content.split('.')
my_file.close()

#print(content)

2.3 Join the PubMed and Squad lists
We keep a small part of SQUAD data(20) in order to run our example.

In [ ]:
train_texts = train_texts[:220]
content=content+train_texts


8737


## ***Step 3:*** Tokenize passages and queries

3.1 BERT-base pretrained model “roberta-large-nli-stsb-mean-tokens” for the tokenization.
Train the data

1.roberta-large-nli-stsb-mean-tokens

2.bert-large-nli-stsb-mean-tokens

3.xlm-r-large-en-ko-nli-ststb

4.monologg/biobert_v1.1_pubmed



https://openprojectrepo.com/project/MartinoMensio-spacy-sentence-bert-python-deep-learning


In [ ]:
#model = SentenceTransformer('roberta-large-nli-stsb-mean-tokens')
#model = SentenceTransformer('bert-large-nli-stsb-mean-tokens')
#model = SentenceTransformer('monologg/biobert_v1.1_pubmed')
model = SentenceTransformer('xlm-r-large-en-ko-nli-ststb')
text_embeddings = model.encode(content, batch_size = 8, show_progress_bar = True)
print(text_embeddings)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

3.2 Load the fine tuned model

In [ ]:
#model = torch.load("/home/evi/Desktop/MedibotModel/robertamodel",map_location=torch.device('cpu'))
#text_embeddings = model.encode(content, batch_size = 8, show_progress_bar = True)

3.3 Save the Model

In [ ]:
# Save model
torch.save(model,"/home/evi/Desktop/MedibotModel/xlm")

## ***Step 4:*** Create Chatbot

4.1 Create some standard answers if the user is greeting.

In [ ]:
#some standard in and outs

GREETING_INPUTS = ("hello", "hi", "whats up","hey")
GREETING_RESPONSES = ["hello", "hi", "whats up","hey"]
def greeting(sentence):
 
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

4.2 

In [ ]:
def index_sort(list_var):
    length = len(list_var)
    list_index = list(range(0, length))
    
    x=list_var
    for i in range(length):
        for j in range(length):
            if x[list_index[i]] > x[list_index[j]]:
                #swap
                temp = list_index[i]
                list_index[i] = list_index[j]
                list_index[j] = temp
    return list_index

4.3 Create a response for the input question

In [ ]:
def response(question):
    
    quest=[]
    quest.append(question)

    answer=''
    user_embeddings = model.encode(quest, batch_size = 8, show_progress_bar = False)
    np.shape(user_embeddings)


    usersim = cosine_similarity(user_embeddings,text_embeddings)
    usersim_sorted = usersim.argsort()
    similarity_scores_list = usersim.flatten()
    index = index_sort(similarity_scores_list)
    index = index[1:]
    response_flag = 0
    j = 0
    for i in range(len(index)):
        if similarity_scores_list[index[i]]>0.3 :
            answer = answer+' '+content[index[i]]+'.'
            response_flag = 1
            j = j+1
        if j > 2:
            break
            
            

    if response_flag == 0:
        answer="I think I need to read more about that..."    
        
    return answer

4.4 Evaluation

In [ ]:
def normalize_text(s):
    """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
    import string, re

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


4.4.2  

Model F1 score represents the model score as a function of precision and recall score. F-score is a machine learning model performance metric that gives equal weight to both the Precision and Recall for measuring its performance in terms of accuracy, making it an alternative to Accuracy metrics (it doesn’t require us to know the total number of observations). It’s often used as a single value that provides high-level information about the model’s output quality. This is a useful measure of the model in the scenarios where one tries to optimize either of precision or recall score and as a result, the model performance suffers. 


Precision
Precision identifies the frequency of correct answers, when the prediction is intent A​. It can be thought of as the answer to the question “Out of all predictions of A, how many were correct?”


Recall
Recall identifies the frequency of detecting A, out of all examples pertaining to A​ in reality. In short, it answers the question “out of all the examples in A, how many were detected?”

https://towardsdatascience.com/classification-metrics-how-to-boost-your-bot-performance-through-data-525349352828


In [ ]:
def compute_f1(myans,ans):

  myans_tokens=normalize_text(myans).split()
  ans_tokens=normalize_text(ans).split()
  print('Tokens of my answer are:',myans_tokens)
  print('Tokens of Medibot','s answer are:',ans_tokens)

  if len(ans_tokens) == 0 or len(myans_tokens) ==0:
    int(ans_tokens == myans_tokens)

  common_tokens = set(ans_tokens) & set(myans_tokens)
  print('The common Tokens are:', len(common_tokens))
  print('The lenght of my answer is:',len(set(myans_tokens)))
  print('The lenght of Medibot','s answer is:',len(set(ans_tokens)))


  if len(common_tokens) == 0:
    print('------')
  else:
    prec= len(common_tokens)/len(set(ans_tokens))
    rec= len(common_tokens)/len(set(myans_tokens))
    f1 = 2* (rec*prec)/(rec+prec)
    print('\nPrecision:',prec,'\n')
    print('Recall:',rec,'\n')
    print('F1:',f1,'\n')



4.4.3 Evaluation of the model

In [ ]:
def evaluation():
    flag=True
    print("\nMediBot: Hello "+name+"! Please enter the question you want to test."+"\n\n"+name+":")
    while(flag==True):
  #  user_choice='no'
        user_response = input()
        user_response=user_response.lower()
        if(user_response!='bye'):
                answer=response(user_response)
                print("\nPlease enter your prediction for this answer\n\n"+name+":")
                prediction=input()
                print("\nMediBot's answer:",end="")
                print(answer)
                #print("\n----------------------------\n",end="")
                #print("\nThe Exact Match and F1 Score are: \n",end="")
                compute_f1(prediction,answer)
                #print(f"F1: {f1}")
                #print("\n----------------------------",end="")

                print("\nDo you want to test another question?\n\n"+name+":")
        else:
            flag=False
            print("\nMediBot: Bye"+name)

4.4.4 Run the chatbot

In [ ]:
def runchatbot():
    flag=True
    print("\nMediBot: Hello "+name+"! Go ahead, you can ask me anything!"+"\n\n"+name+":")
    while(flag==True):
  #  user_choice='no'
        user_response = input()
        user_response=user_response.lower()
        if(user_response!='bye'):
            if(greeting(user_response)!=None):
                print("\nMediBot: "+greeting(user_response)+"\n\n"+name+":")   
            else:
                answer=response(user_response)
                print("\nMediBot: ",end="")
                print(answer)
                print("\nDo you want to ask anything else?\n\n"+name+":")
        else:
            flag=False
            print("\nMediBot: Bye "+name)

4.4.5 Choose to evaluate or run the model

In [ ]:
print("MediBot: Hi I am MediBot. I am an expert on medical staff. What's your name?")
name=input()
print("\nMediBot: Hello "+name+"! Do you want to evaluate or run the chatbot?"+"\n\n"+name+":")
eval_run = input()
if (eval_run!='run'):
    evaluation()
else:
    runchatbot()

MediBot: Hi I am MediBot. I am an expert on medical staff. What's your name?
